In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc

In [2]:
# 节省内存读文件
def reduce_mem_usage(df):
    """
    iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    @param df:
    @return:
    """
    start_mem = df.memory_usage().sum()
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('str')

    end_mem = df.memory_usage().sum()
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
train_user = pd.read_hdf('../input/train_user.h5')
test_user = pd.read_hdf('../input/test_user.h5')

df_user = pd.concat([train_user, test_user])

del train_user, test_user
gc.collect()

66

In [4]:
train_voc = pd.read_hdf('../input/train_voc.h5')
test_voc = pd.read_hdf('../input/test_voc.h5')

df_voc = pd.concat([train_voc, test_voc])

del train_voc, test_voc
gc.collect()

44

In [5]:
train_sms = pd.read_hdf('../input/train_sms.h5')
test_sms = pd.read_hdf('../input/test_sms.h5')

df_sms = pd.concat([train_sms, test_sms])

del train_sms, test_sms
gc.collect()

86

In [6]:
user_phone_no_m = df_user[['phone_no_m']].copy()

In [7]:
df_voc.columns

Index(['phone_no_m', 'opposite_no_m', 'calltype_id', 'start_datetime',
       'call_dur', 'city_name', 'county_name', 'imei_m'],
      dtype='object')

In [8]:
phone_no_m = user_phone_no_m.merge(df_voc, on='phone_no_m', how='left')
phone_no_m = phone_no_m[phone_no_m['start_datetime'].notnull()]
phone_no_m.columns

Index(['phone_no_m', 'opposite_no_m', 'calltype_id', 'start_datetime',
       'call_dur', 'city_name', 'county_name', 'imei_m'],
      dtype='object')

In [9]:
phone_no_m.shape

(985666, 8)

In [10]:
opposite_no_m = user_phone_no_m.merge(df_voc, left_on='phone_no_m', right_on='opposite_no_m', how='left')
opposite_no_m = opposite_no_m[opposite_no_m['start_datetime'].notnull()]
opposite_no_m.columns

Index(['phone_no_m_x', 'phone_no_m_y', 'opposite_no_m', 'calltype_id',
       'start_datetime', 'call_dur', 'city_name', 'county_name', 'imei_m'],
      dtype='object')

In [11]:
opposite_no_m.shape

(289, 9)

In [12]:
opposite_no_m.drop('phone_no_m_y', axis=1, inplace=True)
opposite_no_m.rename(columns={'phone_no_m_x': 'opposite_no_m', 'opposite_no_m': 'phone_no_m'}, inplace=True)

df_voc = pd.concat([phone_no_m, opposite_no_m])
df_voc.shape

(985955, 8)